https://docs.google.com/document/d/1jo1R88kBe7U1PmCbhRvcneADTL0yc3EAcQABYuUEhfc/edit# Markup
De Novo Drug Design https://advances.sciencemag.org/content/4/7/eaap7885
# Reinforcement Learning For Structural Evolution.
This experiement ReLeaSe has two significant parts.
## PART 1 - Use a StackRNN to transform chemical formulas for compounds into a SMILE representation, 
* SMILE is just a String ecnoding for this chemical formula that makes it easier to perform the second part of the experiment.<br>  <br>
* A StackRNN which is a variant of an RNN is used to convert any new chemical formula into the SMILE version.<br>
* An example can be seen in the 2 strings below converted from Carbon monoxide and carbon.<br>
<br>
<b>CO->CHEMBL14688</b><br>
<b>C->CHEMBL17564</b><br>
<br>
* This conversion is especially useful when regarding formulas with double and triple bonds,like below.<br>
<br>
<b>CC(O)=O->CHEMBL539</b><br>
<b>CC(N)=S</b><br>
<br>
* This property is especially useful in this use case since most of the formulas that will be modified are goingto be complex and thus a system like SMILE is used effectively.<br>
## PART 2- Training a generative and predictive model together and use reinforcement learning to bias towards desired properties in the generated compounds.
* In this part of the experiment the valid SMILE strings are fed into a LSTM (Long short term memory network).
* There will be more elaboration about this in the future since there is a lot to say.
* The graphic below is a good description of the experiment flow taken from https://github.com/isayev/ReLeaSE
![image.png](attachment:image.png)

In [6]:
!git clone https://github.com/isayev/ReLeaSE.git
%cd ReLeaSE

fatal: destination path 'ReLeaSE' already exists and is not an empty directory.
/content/ReLeaSE


In [7]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda create -c rdkit -n my-rdkit-env rdkit python=3.6
!source activate my-rdkit-env
!conda install --yes --file conda_requirements.txt
!conda install -c rdkit rdkit nox cairo
!conda install pytorch
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
!pip install  -r pip_requirements.txt

--2020-06-02 23:02:50--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...

In [8]:
!python -m ipykernel install --user --name release --display-name ReLeaSE
import sys
sys.path.append('./release/')


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/local/lib/python3.6/runpy.py", line 142, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/local/lib/python3.6/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.6/site-packages/ipykernel/__init__.py", line 2, in <module>
    from .connect import *
  File "/usr/local/lib/python3.6/site-packages/ipykernel/connect.py", line 13, in <module>
    from IPython.core.profiledir import ProfileDir
  File "/usr/local/lib/python3.6/site-packages/IPython/__init__.py", line 55, in <module>
    from .core.application import Application
  File "/usr/local/lib/python3.6/site-packages/IPython/core/application.py", line 23, in <module>
    from traitlets.config.application import Application, catch_config_error
  File "/usr/loc

In [0]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [0]:
use_cuda = torch.cuda.is_available()

In [19]:
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
from release.stackRNN import StackAugmentedRNN
from release.data import GeneratorData
from release.utils import canonical_smiles

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Explaining RNN's and the StackRNN used in this case
* An RNN is a type of neural network more akin to understanding and predicting sequential information such as stock market quotes.
* This is different from convolutional Neural networks which understand spatial input better.
* In this case the RNN is tasked with taking in past mappings of chemical formulas to SMILE representations and outputting SMILEs for any future inputs.
* Processing text data as input is a popular use for RNN which is what will be done in this case, break the input into sequences of characters
* Leverages the concept of sequential memory.

In [0]:
gen_data_path = './data/testingdata.smi'
tokens = ['<', '>', '#', '%', ')', '(', '+', '-', '/', '.', '1', '0', '3', '2', '5', '4', '7',
          '6', '9', '8', '=', 'A', '@', 'C', 'B', 'F', 'I', 'H', 'O', 'N', 'P', 'S', '[', ']',
          '\\', 'c', 'e', 'i', 'l', 'o', 'n', 'p', 's', 'r', '\n']
gen_data = GeneratorData(training_data_path=gen_data_path, delimiter='\t', 
                         cols_to_read=[0], keep_header=True, tokens=tokens)

In [0]:
def plot_hist(prediction, n_to_generate):
    print("Mean value of predictions:", prediction.mean())
    print("Proportion of valid SMILES:", len(prediction)/n_to_generate)
    ax = sns.kdeplot(prediction, shade=True)
    ax.set(xlabel='Predicted pIC50', 
           title='Distribution of predicted pIC50 for generated molecules')
    plt.show()

In [0]:
def estimate_and_update(generator, predictor, n_to_generate, **kwargs):
    generated = []
    pbar = tqdm(range(n_to_generate))
    for i in pbar:
        pbar.set_description("Generating molecules...")
        generated.append(generator.evaluate(gen_data, predict_len=120)[1:-1])

    sanitized = canonical_smiles(generated, sanitize=False, throw_warning=False)[:-1]
    unique_smiles = list(np.unique(sanitized))[1:]
    smiles, prediction, nan_smiles = predictor.predict(unique_smiles, get_features=get_fp)  
                                                       
    plot_hist(prediction, n_to_generate)
        
    return smiles, prediction

In [24]:
hidden_size = 1500
stack_width = 1500
stack_depth = 200
layer_type = 'GRU'
lr = 0.001
optimizer_instance = torch.optim.Adadelta

my_generator = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                 output_size=gen_data.n_characters, layer_type=layer_type,
                                 n_layers=1, is_bidirectional=False, has_stack=True,
                                 stack_width=stack_width, stack_depth=stack_depth, 
                                 use_cuda=use_cuda, 
                                 optimizer_instance=optimizer_instance, lr=lr)
print(my_generator)

StackAugmentedRNN(
  (stack_controls_layer): Linear(in_features=1500, out_features=3, bias=True)
  (stack_input_layer): Linear(in_features=1500, out_features=1500, bias=True)
  (encoder): Embedding(45, 1500)
  (rnn): GRU(3000, 1500)
  (decoder): Linear(in_features=1500, out_features=45, bias=True)
  (log_softmax): LogSoftmax()
  (criterion): CrossEntropyLoss()
)


In [0]:
model_path = './checkpoints/generator/checkpoint_biggest_rnn'
my_generator.load_model(model_path)

## How a typical RNN works
* Image of a RNN below 
![image.png](attachment:image.png)
* This unrolled network shows how an RNN uses previous information to guess values in the future.
* Activation value from previous timesteps is passed on to the next to help in the prediction, the only timestep that doesn't use previous info is timestep 1.
* A(n) or the activation passed on from the nth layer is calculated A(n) = g(w<sub>aa</sub>a<sup>(n-1)</sup> + w<sub>ax</sub>x<sup>(n-1)</sup> + b<sub>a</sub>)
* In this equation w<sub>aa</sub> represents parameters passed between layers w<sub>ax</sub> represents parameters of the input, b<sub>a</sub> represents a bias and g is the actiavtion function. tanh.Relu
* <hat>yhat</hat><sup>n</sup> or the nework output for a specific layer is computed as $\hat{y}$<sup>n</sup> = g(w<sub>ya</sub>a<sup>(n)</sup>  + b<sub>y</sub>). However this function uses a different activation function as the one which computes the actiavtion passed between layers - Sigmoid.
* It can be seen from these equations that RNN's use a dynamic programming approach to try to deal with sequential data calcualting y(N) from  y(n-1) y(n-2) etc, but the values that are closer in sequence to n tend to have more weight because of the vanishing gradient problem.
* Each time step is a layer.
* One problem here lies in the backpropogation method where in an RNN the earlier layers essentially learn less since the weights are changed less and less as the layers go further back in time.Vanishing gradient problem1
* RNN suffer from short term memory, mitigated by use of LSTM's and GRU's but they can learn long term dependancies using gates, knowing what to add to the hidden state both long term and short term.


## How the StackRNN works
* This experiement uses a Stack Augmented RNN in place of a normal RNN in order to convert formulas into smile strings.
* The goal is to be able to determine the pattern in forming sequences of letters that correspond to valid SMILE strings.
### What is different between a StackRNN and regular RNN's like GRU and LSTM.
* Normal RNN's like GRU's and LSTM's suffer from an inability to count, the most basic example of this is the Dyck language where a Dyck word must have balanced parentheses, but such words would not be able to be generated by a GRU or LSTM since they lack the memory to count parentheses and determine if it is balanced.
* Similarly SMILE strings must have properly matched parentheses with multiple types of brackers, so the RNN that is used for SMILES must have memory.
* Additionally, the rings of molecules muyst have correct valence for all atoms so we need stack memory to count that.
* Another issue of normal RNN's but not GRU's is the short term memory issue that is why in this experiment we use the GRU with a Stack which solves the problem of short term memory and counting.
* GRU's solve the short term memory problem by using gates, that regulate information flow, instead of just passing on the last few bits of information from the last few layers, the gates can learn what sequences of information are important whether they be from recently or a long time ago in the timestamps.
* The way this experiment integrates GRU's and a Stack is shown below.
![image.png](attachment:image.png)* 
* The Stack RNN has a new neuron or cell strucutre on top of the GRU cell, and has 2 additional multiplicative gates which act as a memory stack, and this allows it to learn dependancies in the long term.
* The StackRNN also uses the SoftMax function as loss. This loss function provides a set of probabilities that adds up to one.
* This function is able to map the probability ditribution to the ouput varieties or in this case which values/weights to adjust. 
* This is a good way of reducing the negative impact of the vanishing gradient problem since with this probabilistic approach to weights and value adjustment the lower levels or layers with smaller timestamps will not be neglected in modification.

In [26]:
!git clone --single-branch --branch develop https://github.com/Mariewelt/OpenChem.git

Cloning into 'OpenChem'...
remote: Enumerating objects: 1561, done.
remote: Total 1561 (delta 0), reused 0 (delta 0), pack-reused 1561
Receiving objects: 100% (1561/1561), 108.66 MiB | 31.28 MiB/s, done.
Resolving deltas: 100% (789/789), done.


In [33]:
!pip install -r pip_requirements.txt
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
from release.data import PredictorData
from release.utils import get_desc, get_fp
from mordred import Calculator, descriptors


  Using cached https://files.pythonhosted.org/packages/80/8d/70f26c38ba4bd3bda46791bef71b67bcb7442d6671f66459ad8feaa8eb22/mordred-1.1.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/c6/17/ecd918a004f297955c30b4fffbea100b1606c225dbf0443264012773c3ff/tensorboard-1.10.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/79/43/19c9fee28110cd47f73e6bc596394337fe9f3e5825b4de402bbf30b3beb5/tqdm-4.26.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/54/21/8b2ec99862903a6d3aed62ce156d21d114b8666e669c46d9e54041df9496/xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl
  Could not find a version that satisfies the requirement tensorflow==1.13.0 (from -r pip_requirements.txt (line 5)) (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0rc0, 1.1.0rc1, 1.1.0rc2, 1.1.0, 1.2.0rc0, 1.2.0rc1, 1.2.0rc2, 1.2.0, 1.2.1, 1.3.0rc0, 1.3.0rc1, 1.3.0rc2, 1.3.0, 1.4.0rc0, 1.4.0rc1, 1.4.0, 1.4.1, 1.5.0rc0, 1.5.0rc1, 1.5.0, 1.5.1, 1.6.0rc0, 1.6.0rc1, 1.6.0, 1.7.0r

ModuleNotFoundError: ignored